In [23]:
import os
import logging
import xarray as xr
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

def reset_chunk_encoding(da: xr.Dataset) -> xr.Dataset:
    """resets previous chunk encoding on concatenated zarr stores

    :param da: Dataset
    :return: Dataset with reset chunk encoding
    """
    for x in da.coords:
        if "chunks" in da[x].encoding.keys():
            logging.info(f"deleting chunk encoding for {x}...")
            del da[x].encoding["chunks"]
        if da.coords[x].dtype == object:
            da[x].encoding.clear()

    for var in da:
        if "chunks" in da[var].encoding.keys():
            logging.info(f"deleting chunk encoding for {var}...")
            del da[var].encoding["chunks"]
        if da[x].dtype == object:
            da[x].encoding.clear()
    return da

In [2]:
from dask_jobqueue import SLURMCluster
from distributed import Client

cluster = SLURMCluster(
    # cores=8,  # each worker will have 4 threads
    # processes=1,
    cores=4,  # each worker will have 4 threads
    processes=1,  #48,
    #memory="120GB",  # each worker will have 16GB memory
    memory="24GB",  # each worker will have 16GB memory
    #interface="ib0",  # makes sure workers can communicate with eachother
    walltime="01:30:00",
    job_extra_directives=['--qos=nf'],  # spin up each worker on "nf" queue
    #job_extra_directives=['--qos=np'],  # spin up each worker on "nf" queue
    job_script_prologue=['module load conda; conda activate dask-slurm; export HDF5_USE_FILE_LOCKING=FALSE'],  # ensure correct python environment used
)

In [3]:
# See job script that will be submitted by Dask cluster
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=23G
#SBATCH -t 01:30:00
#SBATCH --qos=nf
module load conda; conda activate dask-slurm; export HDF5_USE_FILE_LOCKING=FALSE
/perm/daep/conda/envs/dask-slurm/bin/python3.8 -m distributed.cli.dask_worker tcp://10.100.192.102:36217 --nthreads 4 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60



In [32]:
client.shutdown()

INFO:distributed.core:Connection to tcp://10.100.192.156:52152 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://10.100.192.156:44423', name: SLURMCluster-2, status: running, memory: 0, processing: 0>
INFO:distributed.core:Removing comms to tcp://10.100.192.156:44423
INFO:distributed.core:Connection to tcp://10.100.192.186:49702 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://10.100.192.186:42239', name: SLURMCluster-0, status: running, memory: 0, processing: 0>
INFO:distributed.core:Removing comms to tcp://10.100.192.186:42239
INFO:distributed.core:Connection to tcp://10.100.192.162:56990 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://10.100.192.162:44475', name: SLURMCluster-5, status: running, memory: 0, processing: 0>
INFO:distributed.core:Removing comms to tcp://10.100.192.162:44475
INFO:distributed.core:Connection to tcp://10.100.192.166:60238 has been closed.
INFO:distributed.scheduler:Remov

In [4]:
# scale cluse to desired number of workers
cluster.scale(jobs=10)

In [8]:
# Watch the dask-workers spinning up
# Wait until dask works have started running
!squeue -u $USER

    JOBID       NAME  USER   QOS    STATE       TIME TIME_LIMIT NODES      FEATURES NODELIST(REASON)
 39441917 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-156
 39441918 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-162
 39441919 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-163
 39441920 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-166
 39441922 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-171
 39441923 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-175
 39441924 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-180
 39441925 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-182
 39441926 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1        (null) ac6-186
 39441927 dask-worke  daep    nf  RUNNING       0:08    1:30:00     1  

In [10]:
# Connect our local client to the dask-workers we have spun up
client = Client(cluster)
client

<Client: 'tcp://10.100.192.102:36217' processes=10 threads=40, memory=223.50 GiB>

In [11]:
import xarray as xr
damet = xr.open_mfdataset("/ec/res4/scratch/daep/ec_training_db_out/ecland*.zarr", engine="zarr", parallel=True,)
damet = damet.rename({var: f"met_{var.lower()}" for var in damet.data_vars})
damet

<xarray.Dataset>
Dimensions:     (time: 4384, x: 187070)
Coordinates:
    lat         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time        (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x           (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables:
    met_ctpf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_snowf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_tair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_e  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_n  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [12]:
damod = xr.open_mfdataset("/ec/res4/scratch/daep/ec_training_db_out/i6aj*.zarr", engine="zarr", parallel=True,)
for var in list(damod.data_vars.keys()):
    if var[0] == "0":
        damod = damod.rename({var: var[1:]})
damod

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/42)
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_geopot  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2023-10-25T21:05 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [24]:
da_train = reset_chunk_encoding(xr.merge([damet, damod.drop(["lat", "lon"])]))
da_train

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/52)
    met_ctpf     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [29]:
# experiment to find an appropriate chunking strategy
da_train.to_array().chunk({"variable":-1, "time":10})#.isel(variable=slice(0,34))

<xarray.DataArray (variable: 52, time: 4384, x: 187070)>
dask.array<rechunk-merge, shape=(52, 4384, 187070), dtype=float32, chunksize=(52, 10, 187070), chunktype=numpy.ndarray>
Coordinates:
    lat       (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon       (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time      (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x         (x) int32 2192 2193 2194 2195 2326 ... 654397 654398 654399 654400
  * variable  (variable) object 'met_ctpf' 'met_lwdown' ... 'tsn_l1' 'tsn_l2'
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [28]:
da_train.chunk({"time":10}).to_zarr("/ec/res4/scratch/daep/ec_training_db/ecland_cy49r2_2020_2022.zarr", consolidated=True)

INFO:distributed.utils_perf:full garbage collection released 11.75 MiB from 280353 reference cycles (threshold: 9.54 MiB)
INFO:distributed.core:Event loop was unresponsive in Scheduler for 6.52s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.


In [31]:
da_test = xr.open_zarr("/ec/res4/scratch/daep/ec_training_db/ecland_cy49r2_2020_2022.zarr")
da_test

<xarray.Dataset>
Dimensions:      (time: 4384, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/52)
    aco2gpp      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF